In [1]:
import pandas as pd
import pymongo
import selenium
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
executable_path = {'executable_path': '../static/geckodriver'}
browser = Browser('firefox', **executable_path, headless=True)
# executable_path = {'executable_path': '../static/chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=True)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
event_df = pd.read_csv("../data/rolling_event_data.csv")
event_df = event_df.drop(columns="Unnamed: 0")
event_df

,event,date,location,event_url
0,UFC Fight Night: Hall vs. Strickland,"July 31, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/320edfb0332b...
1,UFC Fight Night: Sandhagen vs. Dillashaw,"July 24, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/e57f0b06ea39...
2,UFC Fight Night: Makhachev vs. Moises,"July 17, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/40389d39a92f...
3,UFC 264: Poirier vs. McGregor 3,"July 10, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8a0be41c0380...
4,UFC Fight Night: Gane vs. Volkov,"June 26, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/a1d6b308781a...
...,...,...,...,...
566,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, US",http://ufcstats.com/event-details/1c3f5e85b59e...
567,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, US",http://ufcstats.com/event-details/dedc3bb440d0...
568,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, US",http://ufcstats.com/event-details/b60391da771d...
569,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, US",http://ufcstats.com/event-details/1a49e0670dfa...


In [5]:
event_urls = event_df["event_url"].tolist()

In [6]:
fight_urls_df = pd.read_csv("../data/rolling_fight_urls.csv")
fight_urls_df = fight_urls_df.drop(columns="Unnamed: 0")
fight_urls_df

,url
0,http://ufcstats.com/fight-details/8522bc62e8d9...
1,http://ufcstats.com/fight-details/b4622ca7cf0e...
2,http://ufcstats.com/fight-details/e3ecb94228c2...
3,http://ufcstats.com/fight-details/6671b567c4eb...
4,http://ufcstats.com/fight-details/9eef9b796799...
...,...
6192,http://ufcstats.com/fight-details/3b020d4914b4...
6193,http://ufcstats.com/fight-details/d917c8c7461b...
6194,http://ufcstats.com/fight-details/ccee020be2e8...
6195,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [7]:
fight_urls = fight_urls_df["url"].tolist()
fight_urls

['http://ufcstats.com/fight-details/8522bc62e8d942ab',
 'http://ufcstats.com/fight-details/b4622ca7cf0e8506',
 'http://ufcstats.com/fight-details/e3ecb94228c2739f',
 'http://ufcstats.com/fight-details/6671b567c4eb9982',
 'http://ufcstats.com/fight-details/9eef9b7967991296',
 'http://ufcstats.com/fight-details/12ceec9431fb0827',
 'http://ufcstats.com/fight-details/69397bbebdfa41d9',
 'http://ufcstats.com/fight-details/96c51bc6316a4b47',
 'http://ufcstats.com/fight-details/1978b44503c3d9ff',
 'http://ufcstats.com/fight-details/fb9095887dca34c7',
 'http://ufcstats.com/fight-details/214b268e4a92f950',
 'http://ufcstats.com/fight-details/4bb77b1d8efc53a9',
 'http://ufcstats.com/fight-details/1bde9cb65d31cc76',
 'http://ufcstats.com/fight-details/7c3ad06a520b8716',
 'http://ufcstats.com/fight-details/43c391b251337c9b',
 'http://ufcstats.com/fight-details/4911094a4ee26868',
 'http://ufcstats.com/fight-details/dff1dbb8fe1f31cb',
 'http://ufcstats.com/fight-details/1be65a0d2c1383f6',
 'http://u

In [8]:
all_events_url = "http://ufcstats.com/statistics/events/completed?page=all"

In [9]:
browser.visit(all_events_url)
all_events_soup = bs(browser.html, "html.parser")
past_events = all_events_soup.find_all(class_="b-link b-link_style_black")
future_events = all_events_soup.find_all(class_="b-link b-link_style_white")

new_event_urls = []
future_event_urls = []
for event in future_events:
    future_url = event["href"]
    if future_url not in future_event_urls:
        future_event_urls.append(future_url)
        
for url in past_events:
    past_url = url["href"]
    if past_url not in event_urls:
        new_event_urls.append(past_url)
new_event_urls

['http://ufcstats.com/event-details/972197a8c61ac413']

In [10]:
future_event_urls

['http://ufcstats.com/event-details/2f13e4020cea5b38']

In [11]:
future_events_data = []
future_fight_urls = []
for event in future_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    future_events_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in fight_urls:
            future_fight_urls.append(href)
            
future_fight_urls

['http://ufcstats.com/fight-details/f536ba47f006474a',
 'http://ufcstats.com/fight-details/3a9e0fed344fd07d',
 'http://ufcstats.com/fight-details/47230f9b5d7ae037',
 'http://ufcstats.com/fight-details/755d84d5f4aa4dbf',
 'http://ufcstats.com/fight-details/20cb59a505f13d2b',
 'http://ufcstats.com/fight-details/0b47788da6de735e']

In [12]:
future_event_df = pd.DataFrame(future_events_data)
future_event_df["date"] = future_event_df["date"].str.strip("DATE:")
future_event_df["location"] = future_event_df["location"].str.strip("LOCATION:")
future_event_df

,event,date,location,event_url
0,UFC Fight Night: Cannonier vs. Gastelum,"August 21, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/2f13e4020cea...


In [13]:
future_event_df.to_csv("../data/future_event_data.csv")

In [14]:
new_events_data = []
new_fight_urls = []
for event in new_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    new_events_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in fight_urls:
            new_fight_urls.append(href)
            
new_fight_urls

['http://ufcstats.com/fight-details/bf732e22055e8634',
 'http://ufcstats.com/fight-details/980927e7dcc131c0',
 'http://ufcstats.com/fight-details/efe985e626a0d8c0',
 'http://ufcstats.com/fight-details/903d2c361d005b70',
 'http://ufcstats.com/fight-details/962aab0abdd34173',
 'http://ufcstats.com/fight-details/76ece0ee2ef5d240',
 'http://ufcstats.com/fight-details/3d75081ce43774ca',
 'http://ufcstats.com/fight-details/dcd0793ff1a43b68',
 'http://ufcstats.com/fight-details/f2336b7545f5f442',
 'http://ufcstats.com/fight-details/85c5dd604d5b3634',
 'http://ufcstats.com/fight-details/9a678f346946b6a4',
 'http://ufcstats.com/fight-details/cce232c1b1b2e4a0',
 'http://ufcstats.com/fight-details/af9927b5589e708b']

In [15]:
new_event_df = pd.DataFrame(new_events_data)
new_event_df["date"] = new_event_df["date"].str.strip("DATE:")
new_event_df["location"] = new_event_df["location"].str.strip("LOCATION:")
new_event_df

,event,date,location,event_url
0,UFC 265: Lewis vs. Gane,"August 07, 2021","Houston, Texas, US",http://ufcstats.com/event-details/972197a8c61a...


In [16]:
event_frames = [new_event_df, event_df]
rolling_event_df = pd.concat(event_frames)
# rolling_event_df = rolling_event_df.drop(columns="Unnamed: 0")
rolling_event_df

,event,date,location,event_url
0,UFC 265: Lewis vs. Gane,"August 07, 2021","Houston, Texas, US",http://ufcstats.com/event-details/972197a8c61a...
0,UFC Fight Night: Hall vs. Strickland,"July 31, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/320edfb0332b...
1,UFC Fight Night: Sandhagen vs. Dillashaw,"July 24, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/e57f0b06ea39...
2,UFC Fight Night: Makhachev vs. Moises,"July 17, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/40389d39a92f...
3,UFC 264: Poirier vs. McGregor 3,"July 10, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8a0be41c0380...
...,...,...,...,...
566,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, US",http://ufcstats.com/event-details/1c3f5e85b59e...
567,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, US",http://ufcstats.com/event-details/dedc3bb440d0...
568,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, US",http://ufcstats.com/event-details/b60391da771d...
569,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, US",http://ufcstats.com/event-details/1a49e0670dfa...


In [17]:
rolling_event_df.to_csv("../data/rolling_event_data.csv")

In [18]:
fighter_data = []
full_fight_data = []

for fight_url in new_fight_urls:
    
    browser.visit(fight_url)
    print(fight_url)
    
    event_name = browser.find_by_xpath("/html/body/section/div/h2/a").text
    winner_method = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[1]/i[2]").text
    end_round = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[2]").text
    end_time = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[3]").text
    referee = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[5]/span").text
    weight_class = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/i").text
    
    fighter_1_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/div/h3/a").text
    fighter_1_corner_color = "red"
    fighter_1_outcome = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/i").text
    fighter_1_KDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[1]").text
    fighter_1_sig_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[1]").text
    fighter_1_total_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[1]").text
    fighter_1_TDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[1]").text
    fighter_1_sub_atts = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[1]").text
    fighter_1_reversals = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[1]").text
    fighter_1_ctrl_time = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[1]").text
    fighter_1_sig_head = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[4]/p[1]").text
    fighter_1_sig_body = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[5]/p[1]").text
    fighter_1_sig_leg = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[6]/p[1]").text
    fighter_1_sig_distance = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[7]/p[1]").text
    fighter_1_sig_clinch = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[8]/p[1]").text
    fighter_1_sig_ground = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[9]/p[1]").text
    
    fighter_2_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/div/h3/a").text
    fighter_2_corner_color = "blue"
    fighter_2_outcome = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/i").text
    fighter_2_KDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[2]").text
    fighter_2_sig_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[2]").text
    fighter_2_total_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[2]").text
    fighter_2_TDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[2]").text
    fighter_2_sub_atts = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[2]").text
    fighter_2_reversals = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[2]").text
    fighter_2_ctrl_time = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[2]").text
    fighter_2_sig_head = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[4]/p[2]").text
    fighter_2_sig_body = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[5]/p[2]").text
    fighter_2_sig_leg = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[6]/p[2]").text
    fighter_2_sig_distance = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[7]/p[2]").text
    fighter_2_sig_clinch = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[8]/p[2]").text
    fighter_2_sig_ground = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[9]/p[2]").text

    
    fighter_1_info = {"name" : fighter_1_name,
                  "corner" : fighter_1_corner_color,
                  "weight_class": weight_class,
                  "opponent": fighter_2_name,
                  "event": event_name,
                  "outcome" : fighter_1_outcome,
                  "method" : winner_method,
                  "round": end_round,
                  "Time": end_time,
                  "Significant_strikes": fighter_1_sig_str,
                  "Total_strikes": fighter_1_total_str,
                  "KDs": fighter_1_KDs,    
                  "TDs": fighter_1_TDs,
                  "Sub_Atts": fighter_1_sub_atts,
                  "Reversals": fighter_1_reversals,
                  "Control_time": fighter_1_ctrl_time,
                  "Sig_Str_head": fighter_1_sig_head,
                  "Sig_Str_head": fighter_1_sig_head,
                  "Sig_Str_body": fighter_1_sig_body,
                  "Sig_Str_leg": fighter_1_sig_leg,
                  "Sig_Str_distance": fighter_1_sig_distance,
                  "Sig_Str_clinch": fighter_1_sig_clinch,
                  "Sig_Str_ground": fighter_1_sig_ground,}
      
    fighter_2_info = {"name" : fighter_2_name,
                  "corner" : fighter_2_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_1_name,
                  "event" : event_name,
                  "outcome" : fighter_2_outcome,
                  "method" : winner_method,
                  "round": end_round,
                  "Time": end_time,
                  "Significant_strikes": fighter_2_sig_str,
                  "Total_strikes": fighter_2_total_str,
                  "KDs": fighter_2_KDs,
                  "TDs": fighter_2_TDs,
                  "Sub_Atts": fighter_2_sub_atts,
                  "Reversals": fighter_2_reversals,
                  "Control_time": fighter_2_ctrl_time,
                  "Sig_Str_head": fighter_2_sig_head,
                  "Sig_Str_body": fighter_2_sig_body,
                  "Sig_Str_leg": fighter_2_sig_leg,
                  "Sig_Str_distance": fighter_2_sig_distance,
                  "Sig_Str_clinch": fighter_2_sig_clinch,
                  "Sig_Str_ground": fighter_2_sig_ground,}
    
    if fighter_1_outcome == "W":
        winner = "red"
    elif fighter_2_outcome == "W":
        winner = "blue"
    elif fighter_1_outcome == "NC":
        winner = "NC"
    elif fighter_1_outcome == "D":
        winner = "Draw"
    else:
        winner = "None"
        
    fight_info = {"red_name" : fighter_1_name,
                  "blue_name" : fighter_2_name,
                  "weight_class" : weight_class,
                  "event": event_name,
                  "winner" : winner,
                  "method" : winner_method,
                  "round": end_round,
                  "Time": end_time,
                  "red_Significant_strikes": fighter_1_sig_str,
                  "red_Total_strikes": fighter_1_total_str,
                  "red_KDs": fighter_1_KDs,
                  "red_TDs": fighter_1_TDs,
                  "red_Sub_Atts": fighter_1_sub_atts,
                  "red_Reversals": fighter_1_reversals,
                  "red_Control_time": fighter_1_ctrl_time,
                  "red_Sig_Str_head": fighter_1_sig_head,
                  "red_Sig_Str_body": fighter_1_sig_body,
                  "red_Sig_Str_leg": fighter_1_sig_leg,
                  "red_Sig_Str_distance": fighter_1_sig_distance,
                  "red_Sig_Str_clinch": fighter_1_sig_clinch,
                  "red_Sig_Str_ground": fighter_1_sig_ground,
                  "blue_Significant_strikes": fighter_2_sig_str,
                  "blue_Total_strikes": fighter_2_total_str,
                  "blue_KDs": fighter_2_KDs,
                  "blue_TDs": fighter_2_TDs,
                  "blue_Sub_Atts": fighter_2_sub_atts,
                  "blue_Reversals": fighter_2_reversals,
                  "blue_Control_time": fighter_2_ctrl_time,
                  "blue_Sig_Str_head": fighter_2_sig_head,
                  "blue_Sig_Str_body": fighter_2_sig_body,
                  "blue_Sig_Str_leg": fighter_2_sig_leg,
                  "blue_Sig_Str_distance": fighter_2_sig_distance,
                  "blue_Sig_Str_clinch": fighter_2_sig_clinch,
                  "blue_Sig_Str_ground": fighter_2_sig_ground,
                  "url": fight_url}
        
    fighter_data.append(fighter_1_info)    
    fighter_data.append(fighter_2_info)
    full_fight_data.append(fight_info)

http://ufcstats.com/fight-details/bf732e22055e8634
http://ufcstats.com/fight-details/980927e7dcc131c0
http://ufcstats.com/fight-details/efe985e626a0d8c0
http://ufcstats.com/fight-details/903d2c361d005b70
http://ufcstats.com/fight-details/962aab0abdd34173
http://ufcstats.com/fight-details/76ece0ee2ef5d240
http://ufcstats.com/fight-details/3d75081ce43774ca
http://ufcstats.com/fight-details/dcd0793ff1a43b68
http://ufcstats.com/fight-details/f2336b7545f5f442
http://ufcstats.com/fight-details/85c5dd604d5b3634
http://ufcstats.com/fight-details/9a678f346946b6a4
http://ufcstats.com/fight-details/cce232c1b1b2e4a0
http://ufcstats.com/fight-details/af9927b5589e708b


In [19]:
new_fights_df = pd.DataFrame(full_fight_data)
new_fights_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Derrick Lewis,Ciryl Gane,UFC INTERIM HEAVYWEIGHT TITLE BOUT,UFC 265: Lewis vs. Gane,blue,KO/TKO,ROUND: 3,TIME: 4:11,16 of 37,16 of 37,0,0 of 1,0,0,0:08,8 of 24,6 of 11,2 of 2,15 of 36,1 of 1,0 of 0,98 of 122,112 of 136,0,0 of 1,0,0,1:21,50 of 69,16 of 21,32 of 32,67 of 89,16 of 17,15 of 16,http://ufcstats.com/fight-details/bf732e22055e...
1,Jose Aldo,Pedro Munhoz,BANTAMWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,114 of 223,114 of 223,0,0 of 0,0,0,0:00,74 of 168,32 of 47,8 of 8,114 of 223,0 of 0,0 of 0,75 of 179,75 of 179,0,0 of 0,0,0,0:10,35 of 131,3 of 6,37 of 42,71 of 175,4 of 4,0 of 0,http://ufcstats.com/fight-details/980927e7dcc1...
2,Michael Chiesa,Vicente Luque,WELTERWEIGHT BOUT,UFC 265: Lewis vs. Gane,blue,Submission,ROUND: 1,TIME: 3:25,4 of 12,6 of 16,0,1 of 1,2,0,1:01,4 of 12,0 of 0,0 of 0,4 of 12,0 of 0,0 of 0,7 of 12,8 of 13,0,0 of 0,1,0,0:10,2 of 7,1 of 1,4 of 4,7 of 12,0 of 0,0 of 0,http://ufcstats.com/fight-details/efe985e626a0...
3,Tecia Torres,Angela Hill,WOMEN'S STRAWWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,144 of 298,166 of 322,0,1 of 6,0,0,0:16,80 of 217,41 of 52,23 of 29,127 of 274,16 of 22,1 of 2,96 of 188,99 of 191,0,2 of 7,0,0,1:56,64 of 141,29 of 41,3 of 6,73 of 160,22 of 26,1 of 2,http://ufcstats.com/fight-details/903d2c361d00...
4,Song Yadong,Casey Kenney,BANTAMWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Split,ROUND: 3,TIME: 5:00,116 of 257,121 of 263,0,0 of 3,0,0,0:20,66 of 197,34 of 44,16 of 16,110 of 249,6 of 8,0 of 0,78 of 190,85 of 199,0,1 of 5,0,0,0:26,45 of 140,20 of 31,13 of 19,77 of 188,1 of 2,0 of 0,http://ufcstats.com/fight-details/962aab0abdd3...
5,Bobby Green,Rafael Fiziev,LIGHTWEIGHT BOUT,UFC 265: Lewis vs. Gane,blue,Decision - Unanimous,ROUND: 3,TIME: 5:00,143 of 263,143 of 263,0,0 of 2,0,0,0:00,93 of 197,34 of 45,16 of 21,138 of 257,5 of 6,0 of 0,104 of 223,104 of 223,0,0 of 0,0,0,0:08,47 of 132,26 of 44,31 of 47,96 of 212,8 of 11,0 of 0,http://ufcstats.com/fight-details/76ece0ee2ef5...
6,Vince Morales,Drako Rodriguez,BANTAMWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,76 of 196,84 of 204,0,1 of 4,0,0,0:40,46 of 150,26 of 40,4 of 6,69 of 183,7 of 12,0 of 1,55 of 151,60 of 156,0,3 of 5,0,0,1:14,45 of 138,2 of 3,8 of 10,49 of 143,5 of 7,1 of 1,http://ufcstats.com/fight-details/3d75081ce437...
7,Alonzo Menifield,Ed Herman,LIGHT HEAVYWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,93 of 140,94 of 141,0,1 of 1,0,0,0:48,75 of 122,4 of 4,14 of 14,70 of 107,13 of 21,10 of 12,41 of 126,42 of 127,0,0 of 0,0,0,0:01,31 of 115,9 of 10,1 of 1,38 of 119,3 of 7,0 of 0,http://ufcstats.com/fight-details/dcd0793ff1a4...
8,Karolina Kowalkiewicz,Jessica Penne,WOMEN'S STRAWWEIGHT BOUT,UFC 265: Lewis vs. Gane,blue,Submission,ROUND: 1,TIME: 4:32,8 of 12,28 of 36,0,1 of 1,0,0,0:26,3 of 7,0 of 0,5 of 5,4 of 7,0 of 0,4 of 5,12 of 19,21 of 30,0,1 of 1,1,0,3:13,9 of 16,3 of 3,0 of 0,4 of 10,0 of 0,8 of 9,http://ufcstats.com/fight-details/f2336b7545f5...
9,Manel Kape,Ode Osbourne,FLYWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,KO/TKO,ROUND: 1,TIME: 4:44,19 of 29,19 of 29,1,0 of 0,0,0,0:02,14 of 23,2 of 3,3 of 3,16 of 25,0 of 0,3 of 4,18 of 38,18 of 38,0,0 of 0,0,0,0:00,9 of 25,6 of 8,3 of 5,18 of 38,0 of 0,0 of 0,http://ufcstats.com/fight-details/85c5dd604d5b...


In [20]:
all_fights_df = pd.read_csv("../data/rolling_fights_data.csv")
all_fights_df = all_fights_df.drop(columns="Unnamed: 0")
all_fights_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Uriah Hall,Sean Strickland,MIDDLEWEIGHT BOUT,UFC Fight Night: Hall vs. Strickland,blue,Decision - Unanimous,ROUND: 5,TIME: 5:00,107 of 258,122 of 273,0,0 of 0,0,0,0:00,56 of 173,33 of 66,18 of 19,106 of 257,1 of 1,0 of 0,186 of 428,221 of 472,0,4 of 6,0,0,7:02,173 of 414,10 of 11,3 of 3,169 of 400,9 of 12,8 of 16,http://ufcstats.com/fight-details/8522bc62e8d9...
1,Cheyanne Buys,Gloria de Paula,WOMEN'S STRAWWEIGHT BOUT,UFC Fight Night: Hall vs. Strickland,red,KO/TKO,ROUND: 1,TIME: 1:00,11 of 16,11 of 16,1,1 of 1,0,0,0:34,11 of 15,0 of 1,0 of 0,4 of 8,0 of 0,7 of 8,5 of 8,8 of 11,0,0 of 0,0,0,0:00,5 of 8,0 of 0,0 of 0,5 of 8,0 of 0,0 of 0,http://ufcstats.com/fight-details/b4622ca7cf0e...
2,Niklas Stolze,Jared Gooden,WELTERWEIGHT BOUT,UFC Fight Night: Hall vs. Strickland,blue,KO/TKO,ROUND: 1,TIME: 1:08,7 of 13,7 of 13,0,0 of 0,0,0,0:00,4 of 10,2 of 2,1 of 1,7 of 13,0 of 0,0 of 0,14 of 23,14 of 23,1,1 of 1,0,0,0:21,11 of 17,0 of 3,3 of 3,13 of 21,0 of 0,1 of 2,http://ufcstats.com/fight-details/e3ecb94228c2...
3,Collin Anglin,Melsik Baghdasaryan,FEATHERWEIGHT BOUT,UFC Fight Night: Hall vs. Strickland,blue,KO/TKO,ROUND: 2,TIME: 1:50,25 of 51,28 of 57,0,1 of 1,0,0,0:18,14 of 34,9 of 15,2 of 2,19 of 43,6 of 8,0 of 0,44 of 75,51 of 82,1,0 of 0,0,0,1:11,28 of 52,12 of 15,4 of 8,32 of 62,10 of 10,2 of 3,http://ufcstats.com/fight-details/6671b567c4eb...
4,Bryan Barberena,Jason Witt,WELTERWEIGHT BOUT,UFC Fight Night: Hall vs. Strickland,blue,Decision - Majority,ROUND: 3,TIME: 5:00,50 of 113,114 of 178,0,0 of 1,0,1,0:47,50 of 110,0 of 3,0 of 0,40 of 99,4 of 6,6 of 8,35 of 83,56 of 105,1,8 of 18,1,0,5:38,27 of 71,8 of 12,0 of 0,25 of 67,6 of 10,4 of 6,http://ufcstats.com/fight-details/9eef9b796799...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6192,Orlando Wiet,Robert Lucarelli,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 2:50,8 of 12,11 of 15,0,0 of 0,0,0,--,7 of 11,1 of 1,0 of 0,1 of 3,0 of 0,7 of 9,2 of 6,2 of 6,0,1 of 1,1,0,--,1 of 2,0 of 1,1 of 3,2 of 6,0 of 0,0 of 0,http://ufcstats.com/fight-details/3b020d4914b4...
6193,Frank Hamaker,Thaddeus Luster,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 4:52,2 of 3,14 of 15,0,1 of 1,3,1,--,2 of 3,0 of 0,0 of 0,1 of 1,0 of 0,1 of 2,0 of 0,0 of 0,0,0 of 1,0,0,--,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/d917c8c7461b...
6194,Johnny Rhodes,David Levicki,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 12:13,11 of 17,74 of 86,0,1 of 1,0,0,--,9 of 15,1 of 1,1 of 1,1 of 1,1 of 1,9 of 15,4 of 5,95 of 102,0,0 of 0,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,http://ufcstats.com/fight-details/ccee020be2e8...
6195,Patrick Smith,Ray Wizard,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 0:58,1 of 1,1 of 1,0,0 of 1,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1 of 1,2 of 2,0,0 of 0,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [21]:
fight_frames = [new_fights_df, all_fights_df]
rolling_fights_df = pd.concat(fight_frames)
rolling_fights_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Derrick Lewis,Ciryl Gane,UFC INTERIM HEAVYWEIGHT TITLE BOUT,UFC 265: Lewis vs. Gane,blue,KO/TKO,ROUND: 3,TIME: 4:11,16 of 37,16 of 37,0,0 of 1,0,0,0:08,8 of 24,6 of 11,2 of 2,15 of 36,1 of 1,0 of 0,98 of 122,112 of 136,0,0 of 1,0,0,1:21,50 of 69,16 of 21,32 of 32,67 of 89,16 of 17,15 of 16,http://ufcstats.com/fight-details/bf732e22055e...
1,Jose Aldo,Pedro Munhoz,BANTAMWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,114 of 223,114 of 223,0,0 of 0,0,0,0:00,74 of 168,32 of 47,8 of 8,114 of 223,0 of 0,0 of 0,75 of 179,75 of 179,0,0 of 0,0,0,0:10,35 of 131,3 of 6,37 of 42,71 of 175,4 of 4,0 of 0,http://ufcstats.com/fight-details/980927e7dcc1...
2,Michael Chiesa,Vicente Luque,WELTERWEIGHT BOUT,UFC 265: Lewis vs. Gane,blue,Submission,ROUND: 1,TIME: 3:25,4 of 12,6 of 16,0,1 of 1,2,0,1:01,4 of 12,0 of 0,0 of 0,4 of 12,0 of 0,0 of 0,7 of 12,8 of 13,0,0 of 0,1,0,0:10,2 of 7,1 of 1,4 of 4,7 of 12,0 of 0,0 of 0,http://ufcstats.com/fight-details/efe985e626a0...
3,Tecia Torres,Angela Hill,WOMEN'S STRAWWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Unanimous,ROUND: 3,TIME: 5:00,144 of 298,166 of 322,0,1 of 6,0,0,0:16,80 of 217,41 of 52,23 of 29,127 of 274,16 of 22,1 of 2,96 of 188,99 of 191,0,2 of 7,0,0,1:56,64 of 141,29 of 41,3 of 6,73 of 160,22 of 26,1 of 2,http://ufcstats.com/fight-details/903d2c361d00...
4,Song Yadong,Casey Kenney,BANTAMWEIGHT BOUT,UFC 265: Lewis vs. Gane,red,Decision - Split,ROUND: 3,TIME: 5:00,116 of 257,121 of 263,0,0 of 3,0,0,0:20,66 of 197,34 of 44,16 of 16,110 of 249,6 of 8,0 of 0,78 of 190,85 of 199,0,1 of 5,0,0,0:26,45 of 140,20 of 31,13 of 19,77 of 188,1 of 2,0 of 0,http://ufcstats.com/fight-details/962aab0abdd3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6192,Orlando Wiet,Robert Lucarelli,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 2:50,8 of 12,11 of 15,0,0 of 0,0,0,--,7 of 11,1 of 1,0 of 0,1 of 3,0 of 0,7 of 9,2 of 6,2 of 6,0,1 of 1,1,0,--,1 of 2,0 of 1,1 of 3,2 of 6,0 of 0,0 of 0,http://ufcstats.com/fight-details/3b020d4914b4...
6193,Frank Hamaker,Thaddeus Luster,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 4:52,2 of 3,14 of 15,0,1 of 1,3,1,--,2 of 3,0 of 0,0 of 0,1 of 1,0 of 0,1 of 2,0 of 0,0 of 0,0,0 of 1,0,0,--,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/d917c8c7461b...
6194,Johnny Rhodes,David Levicki,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,ROUND: 1,TIME: 12:13,11 of 17,74 of 86,0,1 of 1,0,0,--,9 of 15,1 of 1,1 of 1,1 of 1,1 of 1,9 of 15,4 of 5,95 of 102,0,0 of 0,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,http://ufcstats.com/fight-details/ccee020be2e8...
6195,Patrick Smith,Ray Wizard,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,ROUND: 1,TIME: 0:58,1 of 1,1 of 1,0,0 of 1,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1 of 1,2 of 2,0,0 of 0,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [22]:
rolling_fights_df.to_csv("../data/rolling_fights_data.csv")

In [23]:
rolling_fight_urls = rolling_fights_df["url"]
rolling_fight_url_df = pd.DataFrame(rolling_fight_urls)
rolling_fight_url_df

,url
0,http://ufcstats.com/fight-details/bf732e22055e...
1,http://ufcstats.com/fight-details/980927e7dcc1...
2,http://ufcstats.com/fight-details/efe985e626a0...
3,http://ufcstats.com/fight-details/903d2c361d00...
4,http://ufcstats.com/fight-details/962aab0abdd3...
...,...
6192,http://ufcstats.com/fight-details/3b020d4914b4...
6193,http://ufcstats.com/fight-details/d917c8c7461b...
6194,http://ufcstats.com/fight-details/ccee020be2e8...
6195,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [24]:
rolling_fight_url_df.to_csv("../data/rolling_fight_urls.csv")

In [25]:
future_event_urls

['http://ufcstats.com/event-details/2f13e4020cea5b38']

In [26]:
future_fight_urls = []
future_event_data = []

for event in future_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    future_event_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in future_fight_urls:
            future_fight_urls.append(href)
            
future_fight_urls

['http://ufcstats.com/fight-details/f536ba47f006474a',
 'http://ufcstats.com/fight-details/3a9e0fed344fd07d',
 'http://ufcstats.com/fight-details/47230f9b5d7ae037',
 'http://ufcstats.com/fight-details/755d84d5f4aa4dbf',
 'http://ufcstats.com/fight-details/20cb59a505f13d2b',
 'http://ufcstats.com/fight-details/0b47788da6de735e']

In [27]:
future_fighter_data = []
future_fight_data = []

for fight_url in future_fight_urls:
    
    browser.visit(fight_url)
    print(fight_url)
    
    event_name = browser.find_by_xpath("/html/body/section/div/h2/a").text
    weight_class = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/i").text
    
    fighter_1_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/div/h3/a").text
    fighter_1_corner_color = "red"
    
    fighter_2_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/div/h3/a").text
    fighter_2_corner_color = "blue"
      
    fighter_1_info = {"name" : fighter_1_name,
                  "corner" : fighter_1_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_2_name,
                  "event" : event_name,
                 "outcome" : "tbd",
                 "method" : "tbd",
                  "round": 0,
                  "Time": "0:00",
                  "Significant_strikes": "0 of 0",
                  "Total_strikes": "0 of 0",
                  "KDs": 0,
                  "TDs": "0 of 0",
                  "Sub_Atts": 0,
                  "Reversals": 0,
                  "Control_time": "0:00",
                  "Sig_Str_head": "0 of 0",
                  "Sig_Str_body": "0 of 0",
                  "Sig_Str_leg": "0 of 0",
                  "Sig_Str_distance": "0 of 0",
                  "Sig_Str_clinch": "0 of 0",
                  "Sig_Str_ground": "0 of 0",}
    
    fighter_2_info = {"name" : fighter_2_name,
                  "corner" : fighter_2_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_1_name,
                  "event" : event_name,
                 "outcome" : "tbd",
                 "method" : "tbd",
                  "round": "0",
                  "Time": "0:00",
                  "Significant_strikes": "0 of 0",
                  "Total_strikes": "0 of 0",
                  "KDs": 0,
                  "TDs": "0 of 0",
                  "Sub_Atts": 0,
                  "Reversals": 0,
                  "Control_time": "0:00",
                  "Sig_Str_head": "0 of 0",
                  "Sig_Str_body": "0 of 0",
                  "Sig_Str_leg": "0 of 0",
                  "Sig_Str_distance": "0 of 0",
                  "Sig_Str_clinch": "0 of 0",
                  "Sig_Str_ground": "0 of 0",}
    
    fight_info = {"red_name" : fighter_1_name,
                  "blue_name" : fighter_2_name,
                  "weight_class" : weight_class,
                  "event": event_name,
                  "winner" : "tbd",
                  "method" : "tbd",
                  "round": "0",
                  "Time": "0:00",
                  "red_Significant_strikes": "0 of 0",
                  "red_Total_strikes": "0 of 0",
                  "red_KDs": 0,
                  "red_TDs": "0 of 0",
                  "red_Sub_Atts": 0,
                  "red_Reversals": 0,
                  "red_Control_time": "0:00",
                  "red_Sig_Str_head": "0 of 0",
                  "red_Sig_Str_body": "0 of 0",
                  "red_Sig_Str_leg": "0 of 0",
                  "red_Sig_Str_distance": "0 of 0",
                  "red_Sig_Str_clinch": "0 of 0",
                  "red_Sig_Str_ground": "0 of 0",
                  "blue_Significant_strikes": "0 of 0",
                  "blue_Total_strikes": "0 of 0",
                  "blue_KDs": 0,
                  "blue_TDs": "0 of 0",
                  "blue_Sub_Atts": 0,
                  "blue_Reversals": 0,
                  "blue_Control_time": "0:00",
                  "blue_Sig_Str_head": "0 of 0",
                  "blue_Sig_Str_body": "0 of 0",
                  "blue_Sig_Str_leg": "0 of 0",
                  "blue_Sig_Str_distance": "0 of 0",
                  "blue_Sig_Str_clinch": "0 of 0",
                  "blue_Sig_Str_ground": "0 of 0",
                  "url": fight_url}
        
    future_fighter_data.append(fighter_1_info)    
    future_fighter_data.append(fighter_2_info)
    future_fight_data.append(fight_info)

http://ufcstats.com/fight-details/f536ba47f006474a
http://ufcstats.com/fight-details/3a9e0fed344fd07d
http://ufcstats.com/fight-details/47230f9b5d7ae037
http://ufcstats.com/fight-details/755d84d5f4aa4dbf
http://ufcstats.com/fight-details/20cb59a505f13d2b
http://ufcstats.com/fight-details/0b47788da6de735e


In [28]:
future_fight_df = pd.DataFrame(future_fight_data)
future_fight_df.to_csv("../data/future_fights_data.csv")
future_fight_df

,red_name,blue_name,weight_class,event,winner,method,round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Jared Cannonier,Kelvin Gastelum,MIDDLEWEIGHT BOUT,UFC Fight Night: Cannonier vs. Gastelum,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/f536ba47f006...
1,Clay Guida,Mark Madsen,LIGHTWEIGHT BOUT,UFC Fight Night: Cannonier vs. Gastelum,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/3a9e0fed344f...
2,Parker Porter,Chase Sherman,HEAVYWEIGHT BOUT,UFC Fight Night: Cannonier vs. Gastelum,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/47230f9b5d7a...
3,Vinc Pichel,Austin Hubbard,LIGHTWEIGHT BOUT,UFC Fight Night: Cannonier vs. Gastelum,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/755d84d5f4aa...
4,Cortney Casey,Liana Jojua,WOMEN'S FLYWEIGHT BOUT,UFC Fight Night: Cannonier vs. Gastelum,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/20cb59a505f1...
5,Alexandre Pantoja,Brandon Royval,FLYWEIGHT BOUT,UFC Fight Night: Cannonier vs. Gastelum,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/0b47788da6de...


In [29]:
# UfC fighters are sorted alphabetically by last name with each letter containing a page of fighters
letters = "abcdefghijklmnopqrstuvwxyz"

# Getting URLs of each letter page
urls = []
for letter in letters:
    url = f"http://ufcstats.com/statistics/fighters?char={letter}&page=all"
    urls.append(url)
urls

['http://ufcstats.com/statistics/fighters?char=a&page=all',
 'http://ufcstats.com/statistics/fighters?char=b&page=all',
 'http://ufcstats.com/statistics/fighters?char=c&page=all',
 'http://ufcstats.com/statistics/fighters?char=d&page=all',
 'http://ufcstats.com/statistics/fighters?char=e&page=all',
 'http://ufcstats.com/statistics/fighters?char=f&page=all',
 'http://ufcstats.com/statistics/fighters?char=g&page=all',
 'http://ufcstats.com/statistics/fighters?char=h&page=all',
 'http://ufcstats.com/statistics/fighters?char=i&page=all',
 'http://ufcstats.com/statistics/fighters?char=j&page=all',
 'http://ufcstats.com/statistics/fighters?char=k&page=all',
 'http://ufcstats.com/statistics/fighters?char=l&page=all',
 'http://ufcstats.com/statistics/fighters?char=m&page=all',
 'http://ufcstats.com/statistics/fighters?char=n&page=all',
 'http://ufcstats.com/statistics/fighters?char=o&page=all',
 'http://ufcstats.com/statistics/fighters?char=p&page=all',
 'http://ufcstats.com/statistics/fighter

In [30]:
fighter_urls_df = pd.read_csv("../data/fighter_hrefs.csv")
fighter_urls_df = fighter_urls_df.drop(columns="Unnamed: 0")
fighter_urls_df

,hrefs
0,http://ufcstats.com/fighter-details/93fe7332d1...
1,http://ufcstats.com/fighter-details/15df64c02b...
2,http://ufcstats.com/fighter-details/b361180739...
3,http://ufcstats.com/fighter-details/2f5cbecbbe...
4,http://ufcstats.com/fighter-details/c0ed7b2081...
...,...
3691,http://ufcstats.com/fighter-details/be124bdd60...
3692,http://ufcstats.com/fighter-details/02d808afb9...
3693,http://ufcstats.com/fighter-details/1291dd6b8a...
3694,http://ufcstats.com/fighter-details/523af801b3...


In [32]:
fighter_urls = fighter_urls_df["hrefs"].tolist()
fighter_urls

['http://ufcstats.com/fighter-details/93fe7332d16c6ad9',
 'http://ufcstats.com/fighter-details/15df64c02b6b0fde',
 'http://ufcstats.com/fighter-details/b361180739bed4b0',
 'http://ufcstats.com/fighter-details/2f5cbecbbe18bac4',
 'http://ufcstats.com/fighter-details/c0ed7b208197e8de',
 'http://ufcstats.com/fighter-details/5140122c3eecd307',
 'http://ufcstats.com/fighter-details/c9f6385af6df66d7',
 'http://ufcstats.com/fighter-details/aa6e591c2a2cdecd',
 'http://ufcstats.com/fighter-details/7279654c7674cd24',
 'http://ufcstats.com/fighter-details/1c5879330d42255f',
 'http://ufcstats.com/fighter-details/2620f3eb21c79614',
 'http://ufcstats.com/fighter-details/83b00f7597e5ac83',
 'http://ufcstats.com/fighter-details/a77633a989013265',
 'http://ufcstats.com/fighter-details/79cb2a690b9ba5e8',
 'http://ufcstats.com/fighter-details/1338e2c7480bdf9e',
 'http://ufcstats.com/fighter-details/0e9869d712e81f8f',
 'http://ufcstats.com/fighter-details/ebc5af72ad5a28cb',
 'http://ufcstats.com/fighter-d

In [33]:
# Getting links to each fighter in each letter page
hrefs = []
for url in urls:
    browser.visit(url)
    fighter_soup = bs(browser.html, "html.parser")
    rows = fighter_soup.find_all(class_="b-link b-link_style_black")
    
    
    for cols in rows:
        href = cols["href"]
        if href not in fighter_urls:
            hrefs.append(href)
hrefs

['http://ufcstats.com/fighter-details/ebbf571f349e79d9',
 'http://ufcstats.com/fighter-details/ebbf571f349e79d9',
 'http://ufcstats.com/fighter-details/ebbf571f349e79d9',
 'http://ufcstats.com/fighter-details/4122da4f791b2f4f',
 'http://ufcstats.com/fighter-details/4122da4f791b2f4f',
 'http://ufcstats.com/fighter-details/4122da4f791b2f4f',
 'http://ufcstats.com/fighter-details/36b8f265bcd1b7a4',
 'http://ufcstats.com/fighter-details/36b8f265bcd1b7a4',
 'http://ufcstats.com/fighter-details/36b8f265bcd1b7a4',
 'http://ufcstats.com/fighter-details/d0147e906106cd2e',
 'http://ufcstats.com/fighter-details/d0147e906106cd2e',
 'http://ufcstats.com/fighter-details/d0147e906106cd2e',
 'http://ufcstats.com/fighter-details/16a64f93f6678b7b',
 'http://ufcstats.com/fighter-details/16a64f93f6678b7b',
 'http://ufcstats.com/fighter-details/16a64f93f6678b7b',
 'http://ufcstats.com/fighter-details/8a65fd9ba31fd3f7',
 'http://ufcstats.com/fighter-details/8a65fd9ba31fd3f7',
 'http://ufcstats.com/fighter-d

In [35]:
# Creating dataframe and saving fighter URLs 
new_href_dict = {"hrefs" : hrefs}
new_href_df = pd.DataFrame(new_href_dict)
new_href_df

,hrefs
0,http://ufcstats.com/fighter-details/ebbf571f34...
1,http://ufcstats.com/fighter-details/ebbf571f34...
2,http://ufcstats.com/fighter-details/ebbf571f34...
3,http://ufcstats.com/fighter-details/4122da4f79...
4,http://ufcstats.com/fighter-details/4122da4f79...
5,http://ufcstats.com/fighter-details/4122da4f79...
6,http://ufcstats.com/fighter-details/36b8f265bc...
7,http://ufcstats.com/fighter-details/36b8f265bc...
8,http://ufcstats.com/fighter-details/36b8f265bc...
9,http://ufcstats.com/fighter-details/d0147e9061...


In [36]:
fighter_frames = [new_href_df, fighter_urls_df]
rolling_fighter_df = pd.concat(fighter_frames)
rolling_fighter_df

,hrefs
0,http://ufcstats.com/fighter-details/ebbf571f34...
1,http://ufcstats.com/fighter-details/ebbf571f34...
2,http://ufcstats.com/fighter-details/ebbf571f34...
3,http://ufcstats.com/fighter-details/4122da4f79...
4,http://ufcstats.com/fighter-details/4122da4f79...
...,...
3691,http://ufcstats.com/fighter-details/be124bdd60...
3692,http://ufcstats.com/fighter-details/02d808afb9...
3693,http://ufcstats.com/fighter-details/1291dd6b8a...
3694,http://ufcstats.com/fighter-details/523af801b3...


In [37]:
rolling_fighter_df.to_csv("../data/fighter_hrefs.csv")

In [38]:
all_fighters_data = []
for link in hrefs:
    browser.visit(link)
    print(link)
    
    name = browser.find_by_xpath("/html/body/section/div/h2/span[1]").text
    ufc_record = browser.find_by_xpath("/html/body/section/div/h2/span[2]").text
    nickname = browser.find_by_xpath("/html/body/section/div/p").text

    height = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    weight = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    reach = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[3]").text
    stance = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[4]").text
    DOB = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[5]").text
    SLpM = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[1]").text
    Str_Acc = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[2]").text
    SApM = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[3]").text
    Str_Def = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[1]/ul/li[4]").text
    TD_Avg = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[2]").text
    TD_Acc = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[3]").text
    TD_Def = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[4]").text
    Sub_Avg = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/div[2]/ul/li[5]").text
    
    fighter_data = {"name" : name,
                "ufc_record" : ufc_record,
                "nickname" : nickname,
                "height" : height,
                "reach": reach,    
               "weight" : weight,
               "stance" : stance,
               "DOB" : DOB,
               "SLpM" : SLpM,
               "SApM" : SApM,
               "Str_Acc" : Str_Acc,
               "Str_Def" : Str_Def,
               "TD_Avg" : TD_Avg,
               "TD_Acc" : TD_Acc,
               "TD_Def" : TD_Def,
               "Sub_Avg" : Sub_Avg}
    
    all_fighters_data.append(fighter_data)
    
all_fighters_data

http://ufcstats.com/fighter-details/ebbf571f349e79d9
http://ufcstats.com/fighter-details/ebbf571f349e79d9
http://ufcstats.com/fighter-details/ebbf571f349e79d9
http://ufcstats.com/fighter-details/4122da4f791b2f4f
http://ufcstats.com/fighter-details/4122da4f791b2f4f
http://ufcstats.com/fighter-details/4122da4f791b2f4f
http://ufcstats.com/fighter-details/36b8f265bcd1b7a4
http://ufcstats.com/fighter-details/36b8f265bcd1b7a4
http://ufcstats.com/fighter-details/36b8f265bcd1b7a4
http://ufcstats.com/fighter-details/d0147e906106cd2e
http://ufcstats.com/fighter-details/d0147e906106cd2e
http://ufcstats.com/fighter-details/d0147e906106cd2e
http://ufcstats.com/fighter-details/16a64f93f6678b7b
http://ufcstats.com/fighter-details/16a64f93f6678b7b
http://ufcstats.com/fighter-details/16a64f93f6678b7b
http://ufcstats.com/fighter-details/8a65fd9ba31fd3f7
http://ufcstats.com/fighter-details/8a65fd9ba31fd3f7
http://ufcstats.com/fighter-details/8a65fd9ba31fd3f7


[{'name': 'Jonas Bilharinho',
  'ufc_record': 'RECORD: 8-1-1',
  'nickname': 'SHIVA',
  'height': 'HEIGHT: 5\' 11"',
  'reach': 'REACH: --',
  'weight': 'WEIGHT: 145 lbs.',
  'stance': 'STANCE:',
  'DOB': 'DOB: May 22, 1990',
  'SLpM': 'SLpM: 0.00',
  'SApM': 'SApM: 0.00',
  'Str_Acc': 'Str. Acc.: 0%',
  'Str_Def': 'Str. Def: 0%',
  'TD_Avg': 'TD Avg.: 0.00',
  'TD_Acc': 'TD Acc.: 0%',
  'TD_Def': 'TD Def.: 0%',
  'Sub_Avg': 'Sub. Avg.: 0.0'},
 {'name': 'Jonas Bilharinho',
  'ufc_record': 'RECORD: 8-1-1',
  'nickname': 'SHIVA',
  'height': 'HEIGHT: 5\' 11"',
  'reach': 'REACH: --',
  'weight': 'WEIGHT: 145 lbs.',
  'stance': 'STANCE:',
  'DOB': 'DOB: May 22, 1990',
  'SLpM': 'SLpM: 0.00',
  'SApM': 'SApM: 0.00',
  'Str_Acc': 'Str. Acc.: 0%',
  'Str_Def': 'Str. Def: 0%',
  'TD_Avg': 'TD Avg.: 0.00',
  'TD_Acc': 'TD Acc.: 0%',
  'TD_Def': 'TD Def.: 0%',
  'Sub_Avg': 'Sub. Avg.: 0.0'},
 {'name': 'Jonas Bilharinho',
  'ufc_record': 'RECORD: 8-1-1',
  'nickname': 'SHIVA',
  'height': 'HEIGH

In [39]:
fighter_df = pd.DataFrame(all_fighters_data)
fighter_df

,name,ufc_record,nickname,height,reach,weight,stance,DOB,SLpM,SApM,Str_Acc,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg
0,Jonas Bilharinho,RECORD: 8-1-1,SHIVA,"HEIGHT: 5' 11""",REACH: --,WEIGHT: 145 lbs.,STANCE:,"DOB: May 22, 1990",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
1,Jonas Bilharinho,RECORD: 8-1-1,SHIVA,"HEIGHT: 5' 11""",REACH: --,WEIGHT: 145 lbs.,STANCE:,"DOB: May 22, 1990",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
2,Jonas Bilharinho,RECORD: 8-1-1,SHIVA,"HEIGHT: 5' 11""",REACH: --,WEIGHT: 145 lbs.,STANCE:,"DOB: May 22, 1990",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
3,Kolton Englund,RECORD: 9-3-0,THE WHITE ASSASSIN,"HEIGHT: 5' 8""",REACH: --,WEIGHT: 155 lbs.,STANCE:,"DOB: Jul 09, 1992",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
4,Kolton Englund,RECORD: 9-3-0,THE WHITE ASSASSIN,"HEIGHT: 5' 8""",REACH: --,WEIGHT: 155 lbs.,STANCE:,"DOB: Jul 09, 1992",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
5,Kolton Englund,RECORD: 9-3-0,THE WHITE ASSASSIN,"HEIGHT: 5' 8""",REACH: --,WEIGHT: 155 lbs.,STANCE:,"DOB: Jul 09, 1992",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
6,Abus Magomedov,RECORD: 24-4-1,,"HEIGHT: 6' 2""",REACH: --,WEIGHT: 185 lbs.,STANCE:,"DOB: Sep 02, 1990",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
7,Abus Magomedov,RECORD: 24-4-1,,"HEIGHT: 6' 2""",REACH: --,WEIGHT: 185 lbs.,STANCE:,"DOB: Sep 02, 1990",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
8,Abus Magomedov,RECORD: 24-4-1,,"HEIGHT: 6' 2""",REACH: --,WEIGHT: 185 lbs.,STANCE:,"DOB: Sep 02, 1990",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0
9,Jesse Murray,RECORD: 8-3-0,,"HEIGHT: 6' 3""",REACH: --,WEIGHT: 205 lbs.,STANCE:,"DOB: Jan 26, 1993",SLpM: 0.00,SApM: 0.00,Str. Acc.: 0%,Str. Def: 0%,TD Avg.: 0.00,TD Acc.: 0%,TD Def.: 0%,Sub. Avg.: 0.0


In [40]:
# Stripping out unnecessary text from columns
fighter_df["name"] = fighter_df["name"].str.strip("\n ")
fighter_df["ufc_record"] = fighter_df["ufc_record"].str.strip("RECORD:")
fighter_df["height"] = fighter_df["height"].str.strip("HEIGHT:")
fighter_df["reach"] = fighter_df["reach"].str.strip("REACH:")
fighter_df["weight"] = fighter_df["weight"].str.strip("WEIGHT: lbs.")
fighter_df["stance"] = fighter_df["stance"].str.strip("STANCE:")
fighter_df["DOB"] = fighter_df["DOB"].str.strip("DOB:")
fighter_df["SLpM"] = fighter_df["SLpM"].str.strip("SLpM:")
fighter_df["SApM"] = fighter_df["SApM"].str.strip("SApM:")
fighter_df["Str_Acc"] = fighter_df["Str_Acc"].str.strip("Str. Acc.: %")
fighter_df["Str_Def"] = fighter_df["Str_Def"].str.strip("Str. Def: %")
fighter_df["TD_Avg"] = fighter_df["TD_Avg"].str.strip("TD Avg.:")
fighter_df["TD_Acc"] = fighter_df["TD_Acc"].str.strip("TD Acc.: %")
fighter_df["TD_Def"] = fighter_df["TD_Def"].str.strip("TD Def.: %")
fighter_df["Sub_Avg"] = fighter_df["Sub_Avg"].str.strip("Sub. Avg.:")
fighter_df

,name,ufc_record,nickname,height,reach,weight,stance,DOB,SLpM,SApM,Str_Acc,Str_Def,TD_Avg,TD_Acc,TD_Def,Sub_Avg
0,Jonas Bilharinho,8-1-1,SHIVA,"5' 11""",--,145,,"May 22, 1990",0.00,0.00,0,0,0.00,0,0,0.0
1,Jonas Bilharinho,8-1-1,SHIVA,"5' 11""",--,145,,"May 22, 1990",0.00,0.00,0,0,0.00,0,0,0.0
2,Jonas Bilharinho,8-1-1,SHIVA,"5' 11""",--,145,,"May 22, 1990",0.00,0.00,0,0,0.00,0,0,0.0
3,Kolton Englund,9-3-0,THE WHITE ASSASSIN,"5' 8""",--,155,,"Jul 09, 1992",0.00,0.00,0,0,0.00,0,0,0.0
4,Kolton Englund,9-3-0,THE WHITE ASSASSIN,"5' 8""",--,155,,"Jul 09, 1992",0.00,0.00,0,0,0.00,0,0,0.0
5,Kolton Englund,9-3-0,THE WHITE ASSASSIN,"5' 8""",--,155,,"Jul 09, 1992",0.00,0.00,0,0,0.00,0,0,0.0
6,Abus Magomedov,24-4-1,,"6' 2""",--,185,,"Sep 02, 1990",0.00,0.00,0,0,0.00,0,0,0.0
7,Abus Magomedov,24-4-1,,"6' 2""",--,185,,"Sep 02, 1990",0.00,0.00,0,0,0.00,0,0,0.0
8,Abus Magomedov,24-4-1,,"6' 2""",--,185,,"Sep 02, 1990",0.00,0.00,0,0,0.00,0,0,0.0
9,Jesse Murray,8-3-0,,"6' 3""",--,205,,"Jan 26, 1993",0.00,0.00,0,0,0.00,0,0,0.0


In [41]:
fighter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        18 non-null     object
 1   ufc_record  18 non-null     object
 2   nickname    18 non-null     object
 3   height      18 non-null     object
 4   reach       18 non-null     object
 5   weight      18 non-null     object
 6   stance      18 non-null     object
 7   DOB         18 non-null     object
 8   SLpM        18 non-null     object
 9   SApM        18 non-null     object
 10  Str_Acc     18 non-null     object
 11  Str_Def     18 non-null     object
 12  TD_Avg      18 non-null     object
 13  TD_Acc      18 non-null     object
 14  TD_Def      18 non-null     object
 15  Sub_Avg     18 non-null     object
dtypes: object(16)
memory usage: 2.4+ KB


In [42]:
# convert objects to floats
fighter_df["SLpM"] = fighter_df["SLpM"].astype(float, errors = 'raise')
fighter_df["SApM"] = fighter_df["SApM"].astype(float, errors = 'raise')
fighter_df["Str_Acc"] = fighter_df["Str_Acc"].astype(float, errors = 'raise')
fighter_df["Str_Def"] = fighter_df["Str_Def"].astype(float, errors = 'raise')
fighter_df["TD_Avg"] = fighter_df["TD_Avg"].astype(float, errors = 'raise')
fighter_df["TD_Acc"] = fighter_df["TD_Acc"].astype(float, errors = 'raise')
fighter_df["TD_Def"] = fighter_df["TD_Def"].astype(float, errors = 'raise')
fighter_df["Sub_Avg"] = fighter_df["Sub_Avg"].astype(float, errors = 'raise')
fighter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        18 non-null     object 
 1   ufc_record  18 non-null     object 
 2   nickname    18 non-null     object 
 3   height      18 non-null     object 
 4   reach       18 non-null     object 
 5   weight      18 non-null     object 
 6   stance      18 non-null     object 
 7   DOB         18 non-null     object 
 8   SLpM        18 non-null     float64
 9   SApM        18 non-null     float64
 10  Str_Acc     18 non-null     float64
 11  Str_Def     18 non-null     float64
 12  TD_Avg      18 non-null     float64
 13  TD_Acc      18 non-null     float64
 14  TD_Def      18 non-null     float64
 15  Sub_Avg     18 non-null     float64
dtypes: float64(8), object(8)
memory usage: 2.4+ KB


In [47]:
fighter_df = fighter_df.rename(columns={"Str_Acc":"Str_Acc_pct",
                                       "Str_Def":"Str_Def_pct",
                                       "TD_Acc":"TD_Acc_pct",
                                       "TD_Def":"TD_Def_pct"})
fighter_df = fighter_df.drop_duplicates()
fighter_df

,name,ufc_record,nickname,height,reach,weight,stance,DOB,SLpM,SApM,Str_Acc_pct,Str_Def_pct,TD_Avg,TD_Acc_pct,TD_Def_pct,Sub_Avg
0,Jonas Bilharinho,8-1-1,SHIVA,"5' 11""",--,145,,"May 22, 1990",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Kolton Englund,9-3-0,THE WHITE ASSASSIN,"5' 8""",--,155,,"Jul 09, 1992",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Abus Magomedov,24-4-1,,"6' 2""",--,185,,"Sep 02, 1990",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Jesse Murray,8-3-0,,"6' 3""",--,205,,"Jan 26, 1993",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,Martin Sano,4-2-1,,"5' 11""",--,170,,"May 30, 1991",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,Zhang Minyang,0-0-0,,--,--,--,,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
existing_fighter_df = pd.read_csv("../data/career_fighter_stats.csv")
existing_fighter_df = existing_fighter_df.drop(columns="Unnamed: 0")
existing_fighter_df

,name,ufc_record,nickname,height,reach,weight,stance,DOB,SLpM,SApM,Str_Acc_pct,Str_Def_pct,TD_Avg,TD_Acc_pct,TD_Def_pct,Sub_Avg
0,Tom Aaron,5-3-0,NaN,--,--,155,NaN,"Jul 13, 1978",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
1,Danny Abbadi,4-6-0,THE ASSASSIN,"5' 11""",--,155,Orthodox,"Jul 03, 1983",3.29,4.41,38.0,57.0,0.00,0.0,77.0,0.0
2,David Abbott,10-15-0,TANK,"6' 0""",--,265,Switch,--,1.35,3.55,30.0,38.0,1.07,33.0,66.0,0.0
3,Shamil Abdurakhimov,20-5-0,ABREK,"6' 3""","76""",235,Orthodox,"Sep 02, 1981",2.45,2.45,44.0,58.0,1.23,24.0,47.0,0.2
4,Hiroyuki Abe,8-15-3 (1 NC),ABE ANI,"5' 6""",--,145,Orthodox,--,1.71,3.11,36.0,63.0,0.00,0.0,33.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,Dave Zitanick,5-7-0 (1 NC),NaN,--,--,170,NaN,"Mar 05, 1980",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3692,Alex Zuniga,6-3-0,NaN,--,--,145,NaN,--,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3693,George Zuniga,3-1-0,NaN,"5' 9""",--,185,NaN,--,7.64,5.45,38.0,37.0,0.00,0.0,100.0,0.0
3694,Allan Zuniga,13-1-0,TIGRE,"5' 7""","70""",155,Orthodox,"Apr 04, 1992",3.93,1.80,52.0,61.0,0.00,0.0,57.0,1.0


In [49]:
all_fighter_frames = [fighter_df, existing_fighter_df]
all_fighter_df = pd.concat(all_fighter_frames)
all_fighter_df

,name,ufc_record,nickname,height,reach,weight,stance,DOB,SLpM,SApM,Str_Acc_pct,Str_Def_pct,TD_Avg,TD_Acc_pct,TD_Def_pct,Sub_Avg
0,Jonas Bilharinho,8-1-1,SHIVA,"5' 11""",--,145,,"May 22, 1990",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3,Kolton Englund,9-3-0,THE WHITE ASSASSIN,"5' 8""",--,155,,"Jul 09, 1992",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
6,Abus Magomedov,24-4-1,,"6' 2""",--,185,,"Sep 02, 1990",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
9,Jesse Murray,8-3-0,,"6' 3""",--,205,,"Jan 26, 1993",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
12,Martin Sano,4-2-1,,"5' 11""",--,170,,"May 30, 1991",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,Dave Zitanick,5-7-0 (1 NC),NaN,--,--,170,NaN,"Mar 05, 1980",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3692,Alex Zuniga,6-3-0,NaN,--,--,145,NaN,--,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3693,George Zuniga,3-1-0,NaN,"5' 9""",--,185,NaN,--,7.64,5.45,38.0,37.0,0.00,0.0,100.0,0.0
3694,Allan Zuniga,13-1-0,TIGRE,"5' 7""","70""",155,Orthodox,"Apr 04, 1992",3.93,1.80,52.0,61.0,0.00,0.0,57.0,1.0


In [50]:
all_fighter_df.drop_duplicates()
all_fighter_df

,name,ufc_record,nickname,height,reach,weight,stance,DOB,SLpM,SApM,Str_Acc_pct,Str_Def_pct,TD_Avg,TD_Acc_pct,TD_Def_pct,Sub_Avg
0,Jonas Bilharinho,8-1-1,SHIVA,"5' 11""",--,145,,"May 22, 1990",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3,Kolton Englund,9-3-0,THE WHITE ASSASSIN,"5' 8""",--,155,,"Jul 09, 1992",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
6,Abus Magomedov,24-4-1,,"6' 2""",--,185,,"Sep 02, 1990",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
9,Jesse Murray,8-3-0,,"6' 3""",--,205,,"Jan 26, 1993",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
12,Martin Sano,4-2-1,,"5' 11""",--,170,,"May 30, 1991",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,Dave Zitanick,5-7-0 (1 NC),NaN,--,--,170,NaN,"Mar 05, 1980",0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3692,Alex Zuniga,6-3-0,NaN,--,--,145,NaN,--,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0
3693,George Zuniga,3-1-0,NaN,"5' 9""",--,185,NaN,--,7.64,5.45,38.0,37.0,0.00,0.0,100.0,0.0
3694,Allan Zuniga,13-1-0,TIGRE,"5' 7""","70""",155,Orthodox,"Apr 04, 1992",3.93,1.80,52.0,61.0,0.00,0.0,57.0,1.0


In [51]:
all_fighter_df.to_csv("../data/career_fighter_stats.csv")